# Best-selling video games list

## Web scraping project

Scraping the list of the best-selling video games from Wikipedia + using pandas.

The table will be collected, ready for data cleaning and transforming. Subsequenlty, data will primed for exploration and analysis to gather insights.

## Importing packages

In [3]:
from bs4 import BeautifulSoup
import requests
import re

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_best-selling_video_games'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of best-selling video games - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature

___

## Scraping the table

After checking the url is displayed correctly we can move on with the scraping of the table from the webpage

#### Scraping the table attributes

In [9]:
table = soup.find_all('table')[1]

In [10]:
print(table)

<table class="wikitable sortable plainrowheaders">
<caption><style data-mw-deduplicate="TemplateStyles:r1152813436">.mw-parser-output .sr-only{border:0;clip:rect(0,0,0,0);clip-path:polygon(0px 0px,0px 0px,0px 0px);height:1px;margin:-1px;overflow:hidden;padding:0;position:absolute;width:1px;white-space:nowrap}</style><span class="sr-only">Overview of best-selling video games</span>
</caption>
<tbody><tr>
<th scope="col">Title
</th>
<th scope="col">Sales
</th>
<th scope="col">Series
</th>
<th scope="col">Platform(s)
</th>
<th scope="col">Initial release date
</th>
<th scope="col">Developer(s)<sup class="reference" id="cite_ref-devpub_3-0"><a href="#cite_note-devpub-3"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Publisher(s)<sup class="reference" id="cite_ref-devpub_3-1"><a href="#cite_note-devpub-3"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th class="unsortable" scope="col"><abbr ti

In [11]:
table.find_all('th', scope = 'col')

[<th scope="col">Title
 </th>,
 <th scope="col">Sales
 </th>,
 <th scope="col">Series
 </th>,
 <th scope="col">Platform(s)
 </th>,
 <th scope="col">Initial release date
 </th>,
 <th scope="col">Developer(s)<sup class="reference" id="cite_ref-devpub_3-0"><a href="#cite_note-devpub-3"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
 </th>,
 <th scope="col">Publisher(s)<sup class="reference" id="cite_ref-devpub_3-1"><a href="#cite_note-devpub-3"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
 </th>,
 <th class="unsortable" scope="col"><abbr title="Reference(s)">Ref.</abbr>
 </th>]

In [12]:
headers = table.find_all('th', scope = 'col')

In [13]:
table_attributes = [
    attribute.text.strip() 
    for attribute in headers
]

print(table_attributes)

['Title', 'Sales', 'Series', 'Platform(s)', 'Initial release date', 'Developer(s)[b]', 'Publisher(s)[b]', 'Ref.']


The table attributes above are mostly cleaned, however attributes *Platform(s)*, *Developer(s)[b]* and *Publisher(s)[b]* need some trimming on the end of their string.

We can use the **re** package for this.

In [15]:
def clean_text(text):
    # Regex to remove patterns like '(s)', '[b]', etc.
    # \(.*?\) matches anything in parentheses
    # \[\w\] matches a single character in square brackets
    return re.sub(r'\(.*?\)|\[\w\]', '', text).strip()

In [16]:
table_attributes = [
    clean_text(attribute.text.strip()) 
    for attribute in headers
]

print(table_attributes)

['Title', 'Sales', 'Series', 'Platform', 'Initial release date', 'Developer', 'Publisher', 'Ref.']


---
### Creating the table
Lets import Pandas

In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame(columns = table_attributes)

df

,Title,Sales,Series,Platform,Initial release date,Developer,Publisher,Ref.


#### Scraping the table observations

In [21]:
rows = table.find_all('tr')

print(rows)

[<tr>
<th scope="col">Title
</th>
<th scope="col">Sales
</th>
<th scope="col">Series
</th>
<th scope="col">Platform(s)
</th>
<th scope="col">Initial release date
</th>
<th scope="col">Developer(s)<sup class="reference" id="cite_ref-devpub_3-0"><a href="#cite_note-devpub-3"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Publisher(s)<sup class="reference" id="cite_ref-devpub_3-1"><a href="#cite_note-devpub-3"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th class="unsortable" scope="col"><abbr title="Reference(s)">Ref.</abbr>
</th></tr>, <tr>
<th scope="row"><i><a href="/wiki/Minecraft" title="Minecraft">Minecraft</a></i>
</th>
<td>300,000,000
</td>
<td><i><a href="/wiki/Minecraft_(franchise)" title="Minecraft (franchise)">Minecraft</a></i>
</td>
<td>Multi-platform
</td>
<td><span data-sort-value="000000002011-11-18-0000" style="white-space:nowrap">November 18, 2011</span><sup class="refer

In [22]:
for row in rows:
    row_data = row.find_all('td')
    entry = [data.text.strip() for data in row_data]
    print(entry)

[]
['300,000,000', 'Minecraft', 'Multi-platform', 'November 18, 2011[c]', 'Mojang Studios', 'Mojang Studios / Xbox Game Studios', '[4]']
['200,000,000', 'Grand Theft Auto', 'Multi-platform', 'September 17, 2013', 'Rockstar North', 'Rockstar Games', '[5]']
['82,900,000', 'Wii', 'Wii', 'November 19, 2006', 'Nintendo EAD', 'Nintendo', '[6]']
['75,000,000', 'PUBG Universe', 'Multi-platform', 'December 20, 2017', 'PUBG Studios', 'Krafton', '[7]']
['71,360,000', 'Mario Kart', 'Wii U / Switch', 'May 29, 2014', 'Nintendo EAD / Nintendo EPD (Deluxe)', 'Nintendo', '[d]']
['65,000,000', 'Red Dead', 'Multi-platform', 'October 26, 2018', 'Rockstar Games', 'Rockstar Games', '[5]']
['58,700,000', 'None', 'Multi-platform', 'May 16, 2011', 'Re-Logic', 'Re-Logic / 505 Games', '[10]']
['58,000,000', 'Super Mario', 'Multi-platform', 'September 13, 1985', 'Nintendo R&D4', 'Nintendo', '[e]']
['50,000,000', 'Overwatch', 'Multi-platform', 'May 24, 2016', 'Blizzard Entertainment', 'Blizzard Entertainment', '[1

---
This looks good. Most of the data is here, but the video game names are **missing**.
<br> After further inspection the <u>video game names</u> are considered as <u>table headers</u> which were not detected previously due to the parameters used in the *find_all*


Lets add the <u>video game names</u> to the table 

---

## Appending the data

In [26]:
for row in rows[1:]: # Skipping the first row(table attributes)
    row_data = row.find_all(['th','td']) # first element of each row is a <th>
    observation = [data.text.strip() for data in row_data]
    try: # Used to troubleshoot rows mismatch error
        length = len(df)
        df.loc[length] = observation
    except Exception:
        print(observation)
    

['Human: Fall Flat', 'None', 'Multi-platform', 'July 22, 2016', 'No Brakes Games', 'Curve Digital', '[15]']
['The Witcher 3: Wild Hunt', 'The Witcher', 'Multi-platform', 'May 19, 2015', 'CD Projekt Red', 'CD Projekt', '[16]']
['Diablo III', 'Diablo', 'Multi-platform', 'May 16, 2012', 'Blizzard Entertainment', 'Blizzard Entertainment', '[35]']
['Stardew Valley', 'None', 'Multi-platform', 'February 26, 2016', 'ConcernedApe', 'Chucklefish / ConcernedApe', '[36]']
['Borderlands 2', 'Borderlands', 'Multi-platform', 'September 18, 2012', 'Gearbox Software', '2K Games', '[40]']
['New Super Mario Bros. U / Luigi U / Deluxe', 'Super Mario', 'Wii U / Switch', 'November 18, 2012', 'Nintendo EAD', 'Nintendo', '[m]']
['Cyberpunk 2077', 'Cyberpunk', 'Multi-platform', 'December 10, 2020', 'CD Projekt Red', 'CD Projekt', '[49]']
['Monster Hunter: World', 'Monster Hunter', 'Multi-platform', 'January 18, 2018', 'Capcom', 'Capcom', '[50]']
['Elden Ring', 'None', 'Multi-platform', 'February 25, 2022', 'Fr

After getting a rows mismatch error I inspected the table and found out that these rows share the same sales with other games

**rowspan** in html was used to do this and therefore the other rows did not have sales data.

---

The empty sales entries were filled with *NULL* values.


In [31]:
for row in rows[1:]: # Skipping the first row(table attributes)
    row_data = row.find_all(['th','td']) # first element of each row is a <th>
    observation = [data.text.strip() for data in row_data]
    if len(observation) == len(df.columns) - 1:
        observation.insert(1, None)
    observation_df = pd.DataFrame([observation], columns=df.columns)
    df = pd.concat([df, observation_df], ignore_index=True)
df        

,Title,Sales,Series,Platform,Initial release date,Developer,Publisher,Ref.
0,Minecraft,"300,000,000",Minecraft,Multi-platform,"November 18, 2011[c]",Mojang Studios,Mojang Studios / Xbox Game Studios,[4]
1,Grand Theft Auto V,"200,000,000",Grand Theft Auto,Multi-platform,"September 17, 2013",Rockstar North,Rockstar Games,[5]
2,Wii Sports,"82,900,000",Wii,Wii,"November 19, 2006",Nintendo EAD,Nintendo,[6]
3,PUBG: Battlegrounds,"75,000,000",PUBG Universe,Multi-platform,"December 20, 2017",PUBG Studios,Krafton,[7]
4,Mario Kart 8 / Deluxe,"71,360,000",Mario Kart,Wii U / Switch,"May 29, 2014",Nintendo EAD / Nintendo EPD (Deluxe),Nintendo,[d]
...,...,...,...,...,...,...,...,...
85,Super Mario Bros. 3,"24,430,000",Super Mario,Multi-platform,"October 23, 1988",Nintendo EAD,Nintendo,[q]
86,Call of Duty: Black Ops II,"24,200,000",Call of Duty,Multi-platform,"November 12, 2012",Treyarch,Activision,[43]
87,Hogwarts Legacy,"24,000,000",Wizarding World,Multi-platform,"February 10, 2023",Avalanche Software,Warner Bros. Games,[55]
88,Kinect Adventures!,None,None,Xbox 360,"November 4, 2010",Good Science Studio,Xbox Game Studios,[56]


The most efficient way to populate these *NULLs* is manually.

Adding **code to automate** this process would be more time consuming and require more **debugging** and **upkeep** in the future.

## Adding missing values manually

In [35]:
# Specific Sales values for missing entries in order
missing_sales_values = [
    50000000, 50000000,  # First 2 games
    30000000, 30000000,  # Next 2 
    28000000,            # Next 1 
    26500000,            # Next 1 
    25000000, 25000000, 25000000,  # Next 3 
    24000000             # Last 1 
]

# Find indices of rows with missing Sales values
missing_sales_rows = df[df['Sales'].isna()]

# Fill in missing Sales values based on the provided sequence
for i, index in enumerate(missing_sales_rows.index):
    df.at[index, 'Sales'] = missing_sales_values[i]

In [55]:
df.to_csv(r'C:\Users\conva\Documents\_SortedFiles\CSV\Best_Selling_Video_Games.csv', index = False)